In [27]:
import pandas as pd

In [28]:
df = pd.read_parquet('c:/A1/cleaned_taxi2_2023.parquet')

In [29]:
df.head()

,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,PULocationID,DOLocationID
0,2023-07-01 00:29:59,2023-07-01 00:40:15,1.0,140,263
1,2023-07-01 00:03:25,2023-07-01 00:23:44,1.0,163,163
2,2023-07-01 00:38:29,2023-07-01 00:48:53,1.0,142,262
3,2023-07-01 00:14:16,2023-07-01 00:29:13,1.0,68,24
4,2023-07-01 00:29:32,2023-07-01 00:54:14,1.0,138,186


In [30]:
# Splitting the pickup and dropoff information into separate rows
pickup_df = df[['tpep_pickup_datetime', 'PULocationID', 'passenger_count']].rename(columns={'tpep_pickup_datetime': 'datetime', 'PULocationID': 'LocationID'})
dropoff_df = df[['tpep_dropoff_datetime', 'DOLocationID', 'passenger_count']].rename(columns={'tpep_dropoff_datetime': 'datetime', 'DOLocationID': 'LocationID'})

# Combining the two dataframes
combined_df = pd.concat([pickup_df, dropoff_df], ignore_index=True)

In [31]:
print(combined_df.head())
print(len(combined_df))

             datetime  LocationID  passenger_count
0 2023-07-01 00:29:59         140              1.0
1 2023-07-01 00:03:25         163              1.0
2 2023-07-01 00:38:29         142              1.0
3 2023-07-01 00:14:16          68              1.0
4 2023-07-01 00:29:32         138              1.0
33620292


In [32]:
combined_df['datetime'] = combined_df['datetime'].dt.round('H')
combined_df.info()

C:\Users\Kevin\AppData\Local\Temp\ipykernel_8796\322377648.py:1: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  combined_df['datetime'] = combined_df['datetime'].dt.round('H')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33620292 entries, 0 to 33620291
Data columns (total 3 columns):
 #   Column           Dtype         
---  ------           -----         
 0   datetime         datetime64[us]
 1   LocationID       int32         
 2   passenger_count  float64       
dtypes: datetime64[us](1), float64(1), int32(1)
memory usage: 641.3 MB


In [33]:
combined_df.head()

,datetime,LocationID,passenger_count
0,2023-07-01 00:00:00,140,1.0
1,2023-07-01 00:00:00,163,1.0
2,2023-07-01 01:00:00,142,1.0
3,2023-07-01 00:00:00,68,1.0
4,2023-07-01 00:00:00,138,1.0


In [34]:
combined_df.to_parquet('c:/fomo_data/taxi_2023_2.parquet')

In [35]:
df = pd.read_parquet('c:/fomo_data/taxi_2023_1.parquet')

In [36]:
df = pd.concat([combined_df, df], ignore_index=True)

In [37]:
df.rename(columns={
    'LocationID': 'locationid',
    'datetime': 'transit_timestamp',
    'passenger_count': 'ridership'
}, inplace=True)

In [38]:
combined = pd.read_parquet('c:/fomo_data/sub_taxi_combine.parquet')

In [39]:
df = pd.concat([combined, df], ignore_index=True)

In [40]:
df = df.groupby(['locationid', 'transit_timestamp']).agg({'ridership': 'sum'}).reset_index()

In [41]:
print(len(df))

3857556


In [42]:
df.to_parquet('c:/fomo_data/fully_combined.parquet')